# import libs

In [ ]:
import os
import socket
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import cdist
from GCN import GraphModel as GCN


# import data

In [ ]:
# paths pre-setting
if 'gput' in socket.gethostname() or 'hpc' in socket.gethostname():
    label_root = '/home/sxy841/ERIE/silicone/output'
    model_root = '/home/sxy841/ERIE/silicone/Track-Shuyuan-2023-06-13/videos'
else:
    label_root = '/home/ysy/ERIE/single_psm_manipulation_dataset/output'
    model_root = '/home/ysy/ERIE/videos'

model_name = 'DLC_resnet50_TrackJun13shuffle1_50000'



In [ ]:
# pre setting edges sets
edges = [(0,1),(0,2),(1,3),(2,3),(3,4),(3,5),(4,6),(5,7),
         (8,9),(8,10),(9,11),(10,11),(11,12),(11,13),(12,14),(13,15)]

edge_index = np.zeros((2, len(edges)), dtype=np.int64)

for i, edge in enumerate(edges):
    edge_index[:, i] = edge


In [ ]:
# pre setting training sets
training_sets = ['C_M1_T1_1', 'C_M1_T1_3', 'C_M1_T1_5', 'C_M1_T1_7',
                 'R2_M1_T1_1', 'R2_M1_T1_3', 'R2_M1_T1_5', 'R2_M1_T1_7',
                 'L2_M1_T1_1', 'L2_M1_T1_3', 'L2_M1_T1_5', 'L2_M1_T1_7',
                 'Z2_M1_T1_1', 'Z2_M1_T1_3', 'Z2_M1_T1_5']

# initial arrays
X_train = np.zeros((0, 34))
y_train = np.zeros((0, 3))

for set in tqdm(training_sets):
    # load from files
    labels = np.genfromtxt(os.path.join(
        label_root, set, 'labels_30hz.txt'), delimiter=',')
    coordinates_L = pd.read_hdf(os.path.join(
        model_root, f'{set}_L_h264{model_name}.h5'))
    coordinates_R = pd.read_hdf(os.path.join(
        model_root, f'{set}_R_h264{model_name}.h5'))

    # unify size
    frames = min(len(labels), len(coordinates_L), len(coordinates_R))

    # drop and convert
    coordinates_L = coordinates_L.filter(
        regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]
    coordinates_R = coordinates_R.filter(
        regex='^(?!.*likelihood).*$', axis=1).to_numpy()[:frames]
    labels = labels[:frames, 7:10]
    coordinates = np.hstack(
        (np.zeros((frames, 2)), coordinates_L, coordinates_R))

    X_train = np.vstack((X_train, coordinates))
    y_train = np.vstack((y_train, MinMaxScaler().fit_transform(labels)))

distance_matrices = []
for points in X_train.reshape(-1, 17, 2):
    distances = cdist(points, points)
    distance_matrices.append(distances)

X_train = np.array(distance_matrices)


In [ ]:
X_train.shape, edge_index.shape, y_train.shape

# Trainning and validation

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(X_train, edge_index, y_train, input_dim=17, hidden_dim=64, output_dim=3, batch_size=32, device=device)
model.train(epochs=100, use_tqdm=True, save_loss=True)


# Test

In [ ]:
import json
import matplotlib.pyplot as plt

with open('results/losses_20230727_142246.json', 'r') as f:
    data = json.load(f)

losses = data['losses']

plt.figure(figsize=(10, 6))
plt.plot(range(len(losses)), losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('GCN Training Loss over Epochs')
plt.grid(True)
plt.show()